# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[ 5.2615476 , -4.0487256 ,  4.464928  , -2.9367518 ,  3.5061095 ,
        -4.016832  , -3.463885  ,  6.078449  , -6.9533257 , -1.004144  ],
       [-4.2439985 ,  5.6107073 , -5.669777  , -1.7957246 , -9.255529  ,
         0.7177438 ,  4.4435897 , -2.874715  , -5.090096  ,  9.684121  ],
       [ 4.6749854 ,  8.213466  , -9.075721  ,  9.568374  ,  8.454808  ,
        -1.2327975 ,  3.3903713 , -7.828242  , -0.8454461 ,  0.62885725],
       [-3.008261  ,  4.625961  , -4.483249  ,  2.228457  ,  1.643532  ,
        -2.4505193 , -5.2582016 , -1.6679403 , -7.9857535 ,  2.8311467 ],
       [-5.6072407 ,  2.2695985 , -3.7516532 , -1.8182003 , -5.143028  ,
         7.599364  ,  2.8252366 ,  8.773042  ,  1.6198314 ,  1.1772048 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:40145 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 251.80 GiB


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[-2.8796387e+00,  4.4348764e+00, -4.4264812e+00,  2.3959143e+00,
         1.7286433e+00, -2.4991984e+00, -5.1794515e+00, -1.6771442e+00,
        -8.1329165e+00,  2.6659224e+00, -4.3131094e+00,  5.5827818e+00,
        -5.7318311e+00, -1.7427169e+00, -9.3456125e+00,  7.1365553e-01,
         4.4255500e+00, -2.9118376e+00, -4.9467444e+00,  9.6786413e+00,
         8.4222736e+00, -6.2710242e+00, -6.3596501e+00,  1.9645507e+00,
         4.1715989e+00, -9.1683636e+00,  4.6156683e+00,  8.7916479e+00,
         6.8754416e+00,  2.2288749e+00],
       [-6.9536953e+00, -9.7635880e+00, -6.5648255e+00, -4.3536153e-01,
         6.0998106e+00,  3.7550375e+00, -3.9558537e+00,  6.1595526e+00,
        -1.8599318e+00,  5.0400310e+00, -6.8397551e+00,  1.3435434e+00,
         8.9749022e+00, -9.9621773e-01,  9.6651945e+00,  9.8009663e+00,
        -8.6188364e+00,  5.9978371e+00,  2.2295928e+00, -3.6477711e+00,
         7.0758514e+00, -7.3772264e+00, -5.3214231e+00, -6.9927959e+00,
        -7.9296331e+00,